In [1]:
def startUp():
    print('Holiday Management')
    print('===================')
    
    
    print(f'There are {holidayCount} holidays stored in the system.')

In [2]:
def loadData():
    global holidayCount
    holidayCount = 0
    
    with open('holidayList.json') as f:                     #import holidays from consolidated JSON
        data = json.load(f)
        for item in data:
            holidayList.append(item)                        #create list of holiday dictionaries
            holidayCount+=1                                 
            
    yearDict = {}                            
    for item in holidayList:              
        year = item['date'][0:4]
        yearDict[year] = []                                 #create dictionaries with year string as key, empty list as value
                                                            #Format: yearList = {'2021': [list of holidays], '2022': ...}
    for item in holidayList:  
        holiday = Holiday(item['date'], item['name'])       #create holiday objects from dictionary list
        year = item['date'][0:4]
        try:
            holiday.type = item['type']
            holiday.details = item['details']      
        except:
            pass
        yearDict[year].append(holiday)                      #add holiday object to list, contianed in appropriate dictionary year
        
    return yearDict




    

In [3]:
def addHoliday():
    global yearDict
    
    print('Add a Holiday')
    print('=============')
    name = input('Holiday: ')
    validDate = False
    while validDate == False:
        date = input('Date (YYYY-MM-DD):')                         #validate string is in correct date format
        try:
            dateObj = datetime.strptime(date, '%Y-%m-%d')
            year = date[0:4]
            validDate = True
        except:
            print('Invalid date. Please try again.')
    tempDict={}
    tempDict['date'] = date
    tempDict['name'] = name
    holidayList.append(tempDict)                                   #create new holiday dictionary (for updating JSON file)
    
    newHoliday = Holiday(date, name)                               #create new holiday object
    
    try:
        yearDict[year].append(newHoliday)                          #if the new holiday has a previously unused year, 
    except:                                                        #creates an appropriate dictionary key, and adds holday to list
        yearDict[year] = []                                        #Format: yearList = {'2021': [list of holidays], '2022': ...}
        yearDict[year].append(newHoliday)
    


    
    
    

In [4]:
def message(decorated_fn):
    def inner_fn(*args, **kwargs):
        global removeCount 
        
        removeCount = 0            
        fn_result = decorated_fn(*args, **kwargs)                       #inner function runs removeCount+=1 if successful
        if removeCount > 0:                                             #if successful, print sucess
            print('Success:')
            print('Holiday has been removed from the holiday list')
        else:                                                           #else print error message
            print('Error:')
            print('Holiday not found')
    return inner_fn

In [5]:
@message
def removeHoliday():
    global holidayList                     #modifies list of holiday dictionaries (for printing JSON)
    global yearDict                        #modifies dictionary of holiday objects, by year
    global removeCount                     #modifies count (for tracking function success)
    
    print('Remove a Holiday')
    print('================')
    name = input('Holiday Name: ')
    
    
    newList = []
    
    for holidayDict in holidayList:
        if name != holidayDict['name']:
            newList.append(holidayDict)   #adds non-match holidays to temporary list. Match holidays not added.
        else:
            removeCount+=1
    holidayList = newList                 #the list of dictionaries is updated to equal the temporary list. Matched holidays are removed
    
    for year in yearDict:                         #process repeats for the dictionary of holidays, by year
        tempList = []
        for holidayObj in yearDict[year]:
            if name != holidayObj.name:
                tempList.append(holidayObj)
        yearDict[year] = tempList
  

            
            
    

In [6]:
def saveList():
    print('Saving Holiday List')
    print('===================')
    prompt = input('Are you sure you want to save your changes? [y/n]: ')
    if prompt == 'y':
        filepath = 'holidayList.json'                                           #saves current list of holiday dictionaries as JSON file
        with open(filepath, 'w', encoding='utf-8') as outfile:  
            json.dump(holidayList, outfile, indent=4)
        print('Success: ')
        print('Your changes have been saved')
    else:
        print('Canceled')
        print('Holiday list file save canceled.')

        

In [7]:
def viewHoliday():         
    print('View Holidays')
    print('=================')
    
    validYear = False
    while validYear == False:
        try:
            year = int(input('Which year?: '))            #validates valid year (for date object)
            if year > 0 and year < 9999:
                validYear = True
            else:
                print('Enter a year between 1-9999')      #if not valid year, reprompts user
        except:
            print('Enter an integer')
            
    validWeek = False
    while validWeek == False:
        week = input('Which week? #[1-53, Leave blank for the current week]: ')     #validates valid week
        currentWeek = date.today().isocalendar()[1]
        currentYear = date.today().isocalendar()[0]                                 #identifies current week and year
        if week == '':
            weekNum = currentWeek                                                   #'' returns current week
            validWeek = True
        else:
            try:
                weekNum = int(week)
                if weekNum >= 1 and weekNum <= 53:                                 #if not valid week, reprompts user
                    validWeek = True
                else:
                    print('Enter a week between 1-53')
            except:
                print('Enter an integer')
                
                
    getWeatherDetails = False            
    if weekNum == currentWeek or weekNum == currentWeek + 1:                      #if viewing current Week or next week of the current year, gives option to see weather forcast
        if year == currentYear:
            prompt = input("Would you like to see this week's weather? [y/n]")
            if prompt.lower() == 'y':
                weatherDict = getWeather()                                        #runs function to get weather details from Open Weather API
                getWeatherDetails = True
               
    print('')
    print(f'These are the holidays for {year} week #{weekNum}:')                  
    try:
        count = 0
        for holiday in yearDict[str(year)]:                                      #fetches a list of holiday dictionaries, of only the selected year
            dateObj = datetime.strptime(str(holiday.date), '%Y-%m-%d')           
            holidayWeek = dateObj.isocalendar()[1] 
            match = lambda str1, str2: str1 == str2                              #if the holiday week matches the search week, print results
            if match(holidayWeek, weekNum):
                if getWeatherDetails == True:
                    try:
                        print(f'{holiday}: {weatherDict[holiday.date]}')         #if weather is desired, also prints weather forecast if available
                    except:
                        print(holiday)
                else:
                    print(holiday)
                count+=1       
                     
        if count == 0:                                                          #if no matches were found, let user know
            print('No Holidays Found')
    except:
        print(f'No holidays in {year}')                                         #if search year is not found in dictionary of holidays, error occurs, let user know

In [8]:
def printMenu():
    print('''Holiday Menu\n============\n1. Add a Holiday\n2. Remove a Holiday\n3. Save Holiday List \n4. View Holidays\n5. Exit ''')

In [9]:
def getWeather():
    base_url = "https://api.openweathermap.org/data/2.5/onecall?lat="
    lat = str(44.98)                                                                 #Minneapolis coordinates
    long = str(-93.27)
    exclude = 'current,minutely,hourly,alerts'
    api_key = config.key
    city_name = 'minneapolis'
    complete_url = base_url + lat + '&lon=' + long + "&exclude=" + exclude + '&appid=' + api_key
    
    response = requests.get(complete_url)
    tempDict = json.loads(response.text)

    weekWeatherDict = {}
    for count, forecast in enumerate(tempDict['daily']):
        dateWeather = {}
        weather = forecast['weather'][0]['main']                         

        startingDateObj = date.today()
        targetDateObj = startingDateObj + timedelta(days=count)      #assigns appropriate date to each day's weather forcast
        dateStr = targetDateObj.strftime('%Y-%m-%d')

        weekWeatherDict[dateStr] = weather                           #makes a dictionary of dates as keys, and weather as values
        
    return weekWeatherDict

In [10]:
from datetime import datetime, time, timedelta, date
import json
from dataclasses import dataclass, field
import requests
import config

holidayList = []
removeCount = 0

@dataclass                          #establishes appropriate class for Holidays, with default values for type and details (present in scraped data, but not seed data)
class Holiday:
    date: str
    name: str
    type: str = ''
    details: str = ''
        
    def __str__(self):
        return '%s (%s) %s: ' % (self.name, self.date, self.type)      #updates print function, used in viewHoliday()
    

In [11]:
yearDict = loadData()                                                         #Loads JSON file 
startUp()                                                                     #Displays startup message and Holiday count

isExit = False

printMenu()                                                             
while isExit == False:                                                        #Validates user input to navigate menu
    validNav = False
    while validNav == False:
        userChoice = input('Navigate menu by typing integer 1-5')
        print('')
        try:
            intChoice = int(userChoice)
            if intChoice >=1 and intChoice <= 5:
                validNav = True
            else:
                print('Selected Menu out of bounds')
        except:
            print('Enter a valid integer')

    if intChoice == 1:                                                 #Runs appropriate function depending on user input
        addHoliday()
    elif intChoice == 2:
        removeHoliday()
    elif intChoice == 3:
        saveList()
    elif intChoice == 4:
        viewHoliday()
    elif intChoice == 5:
        isExit = True
    if intChoice != 5:
        print('')
        printMenu()
    else:
        print('Thank you for using Holiday Manager')

Holiday Management
There are 4970 holidays stored in the system.
Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List 
4. View Holidays
5. Exit 


Navigate menu by typing integer 1-5 4



View Holidays


Which year?:  2021
Which week? #[1-53, Leave blank for the current week]:  1



These are the holidays for 2021 week #1:
World Braille Day (2021-01-04) United Nations observance: 
Epiphany (2021-01-06) Christian: 
Orthodox Christmas Day (2021-01-07) Orthodox: 
International Programmers' Day (2021-01-07) Worldwide observance: 
Margaret Thatcher Day (2021-01-10) : 

Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List 
4. View Holidays
5. Exit 


...


Saving Holiday List


Are you sure you want to save your changes? [y/n]:  y


Success: 
Your changes have been saved

Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List 
4. View Holidays
5. Exit 


Navigate menu by typing integer 1-5 5



Thank you for using Holiday Manager
